Types of roles:
- **system**: Sets behavior of assistant. Provides the developer with a way to guide the responses of the assistant. The user will not be aware of this, since they only interact with the assistant. 
- **assistant**: Chat model 
- **user**: You

Example messages based on those roles.
```python
messages = 
[
    {"role": "system",
     "content": "You are a friendly assistant."
    },
    {"role": "user",
     "content": "Tell me a joke."
    },
    {"role": "assistant",
     "content": "Why did the chicken cross the road?"
    },
    {"role": "user",
     "content": "I don't know. Why?" 
    },
    {"role": "assistant",
     "content": "Because the chicken behind it didn't know how to socially distance properly!"     
    }
]
```

In this notebook, we build a conversational chatbot that takes pizza orders. So the messages would look something like this:
```python
messages = 
[
    {"role": "system",
     "content": "You are OrderBot, an automated service to collect orders for a pizza restaurant.."
    },
    {"role": "assistant",
     "content": "Hi. Welcome to our pizza restaurant. Can I take your order?"
    },
    {"role": "user",
     "content": "I want to order a pizza for delivery." 
    },
    {"role": "assistant",
     "content": "Sure! What type of pizza would you like?"     
    }
]


```

In [1]:
import os
import openai
import panel as pn
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_completion_from_messages(messages, model="gpt-4", temperature=1):
    """
    Generate a text completion using the OpenAI API.

    Parameters:
        messages (list): A list of message objects in the conversation. 
            Each message object should have a "role" (either "system", "user", or "assistant") 
            and "content" (the content of the message).
        model (str, optional): The model to use (default is "gpt-4").
        temperature (float, optional): The temperature parameter controlling randomness (default is 0).

    Returns:
        str: The generated text completion.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [3]:
# see note about pn.bind in cell 5 to understand why we use the argument '_' in collect_messages, and then ignore it
def collect_messages(_):
    """
    Collect user messages, generate responses, and update the conversation history.

    Returns:
        pn.Column: A Panel Column containing the conversation history.
    """
    prompt = inp.value_input
    inp.value = ""
    context.append({"role": "user", "content":f"{prompt}"})
    response = get_completion_from_messages(messages =context)
    context.append({"role": "assistant", "content":f"{response}"})
    panels.append(
        pn.Row("User", pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row("OrderBot", pn.pane.Markdown(response, width=600)))
    
    return pn.Column(*panels)

In [4]:
# Enable Panel extension to create interactive web applications
pn.extension()

# Initialize an empty list to store conversation history panels
panels = []

# Define the initial context for the conversation
context = [
    {"role": "system", 
     "content": """You are OrderBot, an automated service to collect orders for a pizza restaurant. 
     You first greet the customer, then collect the order, and then ask if it's a pick up or delivery. 
     You wait to collect the entire order, then summarize it and check for a final time if the customer 
     wants to add anything else. If it is a delivery, you can ask for an address.
     Finally, you collect the payment. Make sure to clarify all options, sizes, and extras to uniquely 
     identify the item from the menue. You respond in a short, very conversational friendly style.
     The menue lists prices from smaller size to larger size and includes:
     cheese pizza 6.00, 9.00, 10.95 
     pepperoni pizza 7.00, 10.00, 12.95 
     eggplant pizza 6.75, 9.75, 11.95 
     fries 3.50, 5.00 
     salad 7.25
     Toppings: 
     extra cheese 2.00
     mushrooms 1.50 
     sausage 3.00
     canadian bacon 3.50
     peppers 1.00
     Drinks:
     coke 1.00, 2.00, 3.00
     sprite 1.00, 2.00, 3.00
     bottled water 3.00
     """
     }
]

In [5]:
# Create a text input widget for user input
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text")

# Create a button widget for initiating the conversation
button_conversation = pn.widgets.Button(name="Chat!")

# Bind the collect_messages function to the button click event
# Note: When you use pn.bind, it implicitly passes an argument (an event) to the bound function. 
# If your function does not expect any arguments, you should define it to accept an argument and then ignore it.
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create a Panel Column to organize the input, button, and conversation history
dashboard = pn.Column(
    pn.panel(interactive_conversation, loading_indicator=True),
    inp,
    pn.Row(button_conversation),
)

# Display the dashboard
dashboard

Column
    [0] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)
    [1] TextInput(placeholder='Enter text')
    [2] Row
        [0] Button(name='Chat!')

In [6]:
# Record the order submit to order system
messages = context.copy()
messages.append(
    {"role": "system",  # could also be "user"
     "content": """create a json summary of the previous food order. Itemize the price for each item. 
     The fields should be 
     1. pizza, include sides, 
     2. list of toppings, 
     3. list of drinks, 
     4. list of sides, 
     5. total price"""
     }
)
# Temperature is zero because we want the output to be predictable 
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni",
    "size": "medium",
    "price": 10.00,
    "toppings": [
      {
        "type": "peppers",
        "price": 1.00
      }
    ]
  },
  "drinks": [
    {
      "type": "coke",
      "size": "medium",
      "price": 2.00
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "small",
      "price": 3.50
    }
  ],
  "total_price": 16.50
}
